So we have this following dataset and we are going to fit this with a second order poynomial

In [2]:
x = [6.230825,6.248279,6.265732]
y = [0.312949,0.309886,0.306639472]

As we are fitting 3 points with a 2 degree polynomial, any original x value should exactly poroduce the same corresponding y value. Here, we are going to verify for x[2], which should produce the exact value of y[2] 

In [3]:
toCheck = 6.265732
result = 0.306639472

now we are going to apply different methods in different way to fit the data and then verify the goodness of the fit. A correct method will produce 0% error

In [4]:
def evaluateValue(coeff,x):
    c,b,a = coeff
    val = a+b*x+c*x**2 
    act = 0.306639472
    error=  np.abs(act-val)*100/act
    print "Value = {:.9f} Error = {:.2f}%".format(val,error)

lets start with the solution method using soving system of linear equation using Cramer's rule and check its solution

In [5]:
def determinantAlt(a,b,c,d,e,f,g,h,i):
    return a*e*i - a*f*h - b*d*i +b*g*f + c*d*h - c*e*g

a = b = c = d = e = m = n = p = 0
a = len(x)
for i,j in zip(x,y):
        b += i
        c += i**2
        d += i**3
        e += i**4
        m += j
        n += j*i
        p += j*i**2
det = determinantAlt(a,b,c,b,c,d,c,d,e)
c0 = determinantAlt(m,b,c,n,c,d,p,d,e)/det
c1 = determinantAlt(a,m,c,b,n,d,c,p,e)/det
c2 = determinantAlt(a,b,m,b,c,n,c,d,p)/det
# using explicit determinat math, without brackets, gives 91.03% error, 
# due to float precision error for large and small value multiplication
evaluateValue([c2,c1,c0], toCheck)

Value = 0.585786477 Error = 91.03%


Wohhh !!! 90.73 % error. Did I typed the formula wrong ! Nope. Then what is going on. Let's try the same just by rearranging the formula a little bit

In [6]:
def determinant(a,b,c,d,e,f,g,h,i):
    return a*(e*i - f*h) - b*(d*i - g*f) + c*(d*h - e*g)


a = b = c = d = e = m = n = p = 0
a = len(x)
for i,j in zip(x,y):
        b += i
        c += i**2
        d += i**3
        e += i**4
        m += j
        n += j*i
        p += j*i**2
det = determinant(a,b,c,b,c,d,c,d,e)
c0 = determinant(m,b,c,n,c,d,p,d,e)/det
c1 = determinant(a,m,c,b,n,d,c,p,e)/det
c2 = determinant(a,b,m,b,c,n,c,d,p)/det


evaluateValue([c2,c1,c0], toCheck)


Value = 0.308333580 Error = 0.55%


Huh! Not the mathematics but there's something wrong with the programming here. So, what's the problem

To understand whats going on you have to learn something called error propagation and how computer does floating point operations. (For an in depth discussion read https://www.itu.dk/~sestoft/bachelor/IEEE754_article.pdf). You see computer can't store floating point numbers accurately all the time and sometimes it introduces a (tiny) error to fit the number in limited bits (https://www.h-schmidt.net/FloatConverter/IEEE754.html). And when you are working with very small number this can effect how the error propagates.
Now check the values the determinant quantities,

In [7]:
print (a,b,c,d,e,m,n,p)
print determinant(a,b,c,b,c,d,c,d,e)
print determinantAlt(a,b,c,b,c,d,c,d,e)

(3, 18.744836, 117.12356813829001, 731.8283056811686, 4572.738547313946, 0.9294744720000001, 5.807505391292503, 36.28641270376207)
1.09134035142e-10
2.32830643654e-10


See, theres a considerable difference in order of values of the variable and the determinant is in the order of `e-10`. This very small number calculations affects how the error propagates. Though both the determinant formulas are essentially same and they should produce the exact result, due to this error propagation we get slightly different result. And when these small number is used in division, they give very different result

One way to minimize this kind of error is to make the numbers in calculation, of same order. This can be done by normalizing the input, 

In [8]:
me = sum(x) / len(x)
xx = [pt - me for pt in x]
toCheck = x[2] - me

Now if we calculate the quantities,  

In [9]:
a = b = c = d = e = m = n = p = 0
a = len(xx)
for i,j in zip(xx,y):
        b += i
        c += i**2
        d += i**3
        e += i**4
        m += j
        n += j*i
        p += j*i**2
det = determinantAlt(a,b,c,b,c,d,c,d,e)
c0 = determinantAlt(m,b,c,n,c,d,p,d,e)/det
c1 = determinantAlt(a,m,c,b,n,d,c,p,e)/det
c2 = determinantAlt(a,b,m,b,c,n,c,d,p)/det
evaluateValue([c2,c1,c0], toCheck)


a = b = c = d = e = m = n = p = 0
a = len(xx)
for i,j in zip(xx,y):
        b += i
        c += i**2
        d += i**3
        e += i**4
        m += j
        n += j*i
        p += j*i**2
det = determinant(a,b,c,b,c,d,c,d,e)
c0 = determinant(m,b,c,n,c,d,p,d,e)/det
c1 = determinant(a,m,c,b,n,d,c,p,e)/det
c2 = determinant(a,b,m,b,c,n,c,d,p)/det


evaluateValue([c2,c1,c0], toCheck)

Value = 0.306639472 Error = 0.00%
Value = 0.306639472 Error = 0.00%
